The purpose of this notebook is to analyze the LP we used to understand to introduce duals, to understand its dual, to understand how changing it, changes its dual, to flag an error we (I!) made in class, and to learn about the SARHSUp command in Gurobi.

We'll begin by just setting up the LP we saw in class.

In [1]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 14)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

Using license file /Users/dfreund/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x898fc204
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       2    6.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  6.200000000e+01


In [2]:
# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

4.0
10.0
0.0


As we can see, the optimal objective is 62 which does not match the upper bound we found in class. Is strong duality violated? No; the upper bound we identified (based on 3 times the first constraint and 1 times the second) was not actually the strongest possible one. Let's ask Python for the dual variables.

In [3]:
# print values of the dual solution corresponding to our constraints
print('duals:')
for c in [c1,c2,c3,c4]: print(c.pi)

duals:
3.0
0.0
0.0
2.0


Based on this, we can check the dual solution Gurobi finds: 3 times the first constraint plus 2 times the fourth constraint. We can first check it's a feasible dual solution: we get 3x from the first constraint, we get 3y from the first and 2y from the fourth (together 5y which is what we need), and we get 3z from the first and 2z from the fourth constraint (together 5z which is more than what we need). So, it is feasible; and 3 times 14 plus 2 times 10 (right-hand sides of the first and fourth constraint) gives us 62 as opposed to the 64 we identified in class.

Now, what happens if we change our 14 to 16 as in class?

In [4]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 16)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

print('duals:')
# & values of the dual solution corresponding to constraints
for c in [c1,c2,c3,c4]: print(c.pi)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xd9b1593b
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.01s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       2    6.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  6.800000000e+01
6.0
10.0
0.0
duals:
3.0
0.0
0.0
2.0


The dual solution remains the same, and its objective increases by the dual variable of the first constraint (3) multiplied by the increase in that constraint (16-14=2), so we get a new objective that equals $old\; objective+3\times 2=62+6=68$, which matches the new optimal primal solution (see above: optimal objective 6.800... e+01).

Now, if instead we increased the RHS of the second constraint, nothing much would change: the same objective (62) as before the increase is still feasible (clear?), and the same dual solution as before still gives the same upper bound.

In [5]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 14)
c2=m.addConstr(2*y+z <= 32)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

print('duals:')
# & values of the dual solution corresponding to constraints
for c in [c1,c2,c3,c4]: print(c.pi)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xee1245a9
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+01   3.994500e+00   0.000000e+00      0s
       3    6.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  6.200000000e+01
4.0
10.0
0.0
duals:
3.0
0.0
0.0
2.0


Now, if instead we increased the RHS of the first constraint from 14 to 19 (instead of to 16), we find that things do change in somewhat interesting ways.

In [6]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 19)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

print('duals:')
# & values of the dual solution corresponding to constraints
for c in [c1,c2,c3,c4]: print(c.pi)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x549a2fb1
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.01s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       3    6.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  6.800000000e+01
6.0
10.0
0.0
duals:
0.0
0.0
1.5
5.0


So, here the objective does not increase by 3 (old dual variable of first constraint) times 5 (increase of that constraint) which would equal 15 (i.e., we'd get 62+15), but rather by just 6. The reason is that beyond a certain point (in fact, beyond 16) increasing the RHS of the first constraint no longer yields incremental value. For example, if we changed the first constraint's RHS to 16.001, we already get a dual value of 0 for that constraint (see below). So one needs to be very careful when considering the dual value of 3 at 16: reducing the RHS to 15 would indeed reduce the objective by 3, but increasing the RHS to 17 would not increase the objective by 3. 

In [7]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 16.001)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

print('duals:')
# & values of the dual solution corresponding to constraints
for c in [c1,c2,c3,c4]: print(c.pi)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xf330dc10
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       3    6.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  6.800000000e+01
6.0
10.0
0.0
duals:
0.0
0.0
1.5
5.0


Now, do we need to resolve the LP at various different RHS values for the first constraint to find that it is at 16 that the dual solution changes from (3,0,0,2) to (0,0,1.5,5)? That'd be a lot of effort. Instead, we can use SARHSUp (https://www.gurobi.com/documentation/9.1/refman/sarhsup.html): we solve the LP as we did initially below.

In [3]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 14)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x898fc204
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       2    6.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  6.200000000e+01


Then we consider constraint 1 and call c1.SARHSUp:

In [4]:
c1.SARHSUp

16.0

Which tells us how far we can increase the first constraint's RHS before the optimal dual solution changes. Similarly, c1.SARHSLow would give us how far we could decrease the RHS.

In [6]:
c1.SARHSLow

10.0

Just to sanity-check, let's resolve the LP and read off the duals with the first constraint having a RHS of 10.1 and 9.9:

In [8]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 10.1)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

print('duals:')
# & values of the dual solution corresponding to constraints
for c in [c1,c2,c3,c4]: print(c.pi)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x63c58ecf
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       2    5.0300000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  5.030000000e+01
0.09999999999999964
10.0
0.0
duals:
3.0
0.0
0.0
2.0


In [9]:
import gurobipy as gp

# First, define a variable that contains the optimization model
m = gp.Model("dual_example")

# define our three decision variables x,y,z, each non-negative
variables = []
x,y,z = m.addVar(lb=0), m.addVar(lb=0), m.addVar(lb=0)

# next, we can set our objective: 3x+5y+4z
m.setObjective(3*x+5*y+4*z, gp.GRB.MAXIMIZE) 

# next, we define, and store, our constraints 
# in the same order as in class 
c1=m.addConstr(x+y+z <= 9.9)
c2=m.addConstr(2*y+z <= 22)
c3=m.addConstr(2*x+2*z <= 12)
c4=m.addConstr(y+z<=10)

m.optimize()

# print all the decision variables' values
for variable in [x,y,z]: print(variable.x)

print('duals:')
# & values of the dual solution corresponding to constraints
for c in [c1,c2,c3,c4]: print(c.pi)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 4 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x5149d3c0
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+31   8.500000e+30   1.200000e+01      0s
       1    4.9500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  4.950000000e+01
0.0
9.9
0.0
duals:
5.0
0.0
0.0
0.0
